In [15]:
import cv2 as cv
import numpy as np
import glob
import matplotlib.pyplot as plt

# --------------------- config ---------------------
DEBUG = False  # turn off verbose output by default
RATIO_TEST = 0.75
RANSAC_THRESH = 3.0
RESIZE_MAX = 900  # downscale for speed
MAX_CANVAS = 8000  # clamp max canvas width/height in pixels
# --------------------------------------------------

def show_img(img, title=""):
    plt.figure(figsize=(10,6))
    if img.ndim == 2:
        plt.imshow(img, cmap="gray")
    else:
        plt.imshow(cv.cvtColor(img.astype(np.uint8), cv.COLOR_BGR2RGB))
    plt.title(title); plt.axis("off"); plt.show()

def draw_polys_debug(canvas_size, images, H_to_anchor, canvas_T):
    vis = np.zeros((canvas_size[1], canvas_size[0], 3), np.uint8)
    for i, (img, H) in enumerate(zip(images, H_to_anchor)):
        h, w = img.shape[:2]
        cs = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
        wc = cv.perspectiveTransform(cs, canvas_T @ H).reshape(-1,2).astype(np.int32)
        cv.polylines(vis, [wc.reshape(-1,1,2)], True, (0,255,0), 2)
        cx, cy = wc.mean(axis=0).astype(int)
        cv.putText(vis, str(i), (cx, cy), cv.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    show_img(vis, "Warped footprints with IDs")

def imread_color(p):
    img = cv.imread(str(p), cv.IMREAD_COLOR)
    if img is None:
        raise RuntimeError(f"Failed to read {p}")
    if RESIZE_MAX is not None:
        h, w = img.shape[:2]
        s = RESIZE_MAX / max(h, w)
        if s < 1.0:
            img = cv.resize(img, (int(w*s), int(h*s)), interpolation=cv.INTER_AREA)
    return img

def to_gray(img):
    return cv.cvtColor(img, cv.COLOR_BGR2GRAY)


In [16]:
def detect_and_match(imgA, imgB, idxA=None, idxB=None, visualize=False):
    sift = cv.SIFT_create(nfeatures=2500, contrastThreshold=0.02, edgeThreshold=10)
    grayA, grayB = to_gray(imgA), to_gray(imgB)
    kA, dA = sift.detectAndCompute(grayA, None)
    kB, dB = sift.detectAndCompute(grayB, None)

    if dA is None or dB is None or len(kA) < 8 or len(kB) < 8:
        return None, None, None, None

    flann = cv.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=100))
    raw = flann.knnMatch(dA, dB, k=2)
    good = [m for m,n in raw if m.distance < RATIO_TEST * n.distance]

    if len(good) < 8:
        return None, None, None, None

    ptsA = np.float32([kA[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    ptsB = np.float32([kB[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    H, mask = cv.findHomography(ptsA, ptsB, cv.RANSAC, RANSAC_THRESH)

    if idxA is not None and idxB is not None:
        inliers = int(mask.sum()) if mask is not None else 0
        if 'DEBUG' in globals() and DEBUG:
            print(f"[match] {idxA} -> {idxB} : {len(good)} matches, {inliers} inliers")

    if visualize or ('DEBUG' in globals() and DEBUG):
        matched_vis = cv.drawMatches(imgA, kA, imgB, kB, good[:50], None,
                                     flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        show_img(matched_vis, f"Matches {idxA}->{idxB}")

    return H, mask, ptsA, ptsB


In [17]:

def _normalize_H(H):
    if H is None:
        return None
    if abs(H[2,2]) < 1e-12:
        return H
    return H / H[2,2]

def accumulate_homographies(images):
    """
    Build transforms that map each image into the center (anchor) frame.
    Robust composition with None-guarding so we never matmul with None.
    """
    N = len(images)
    if N == 0:
        return [], 0
    center = N // 2

    H_to_anchor = [None] * N
    H_to_anchor[center] = np.eye(3, dtype=np.float64)

    # ---- forward: center -> right ----
    # For pair (i, i+1): estimate H(i+1 <- i). To map (i+1) to anchor:
    # H_to_anchor[i+1] = H_to_anchor[i] @ inv(H(i+1 <- i))
    for i in range(center, N - 1):
        H, _, _, _ = detect_and_match(images[i + 1], images[i], i+1, i, visualize=False)  # H: (i+1) <- i
        H = _normalize_H(H)
        if H is None:
            # propagate last known pose
            H_to_anchor[i + 1] = H_to_anchor[i] if H_to_anchor[i] is not None else np.eye(3, dtype=np.float64)
            if 'DEBUG' in globals() and DEBUG:
                print(f"[warn] No reliable H between {i} and {i+1} (right). Pose copied.")
        else:
            base = H_to_anchor[i] if H_to_anchor[i] is not None else np.eye(3, dtype=np.float64)
            try:
                Hinv = np.linalg.inv(H)
            except np.linalg.LinAlgError:
                Hinv = np.eye(3, dtype=np.float64)
                if 'DEBUG' in globals() and DEBUG:
                    print(f"[warn] H between {i} and {i+1} singular; using identity.")
            H_to_anchor[i + 1] = base @ Hinv

    # ---- backward: center -> left ----
    # For pair (i-1, i): estimate H(i-1 -> i). To map (i-1) to anchor:
    # H_to_anchor[i-1] = H_to_anchor[i] @ H(i-1 -> i)
    for i in range(center, 0, -1):
        H, _, _, _ = detect_and_match(images[i - 1], images[i], i-1, i, visualize=False)  # H: (i-1)->i
        H = _normalize_H(H)
        if H is None:
            H_to_anchor[i - 1] = H_to_anchor[i] if H_to_anchor[i] is not None else np.eye(3, dtype=np.float64)
            if 'DEBUG' in globals() and DEBUG:
                print(f"[warn] No reliable H between {i-1} and {i} (left). Pose copied.")
        else:
            base = H_to_anchor[i] if H_to_anchor[i] is not None else np.eye(3, dtype=np.float64)
            H_to_anchor[i - 1] = base @ H

    return H_to_anchor, center



In [18]:
PADDING = 80  # safety margin in pixels

def compute_canvas_bounds(images, H_to_anchor):
    corners = []
    polys   = []
    for img, H in zip(images, H_to_anchor):
        h, w = img.shape[:2]
        cs = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
        wc = cv.perspectiveTransform(cs, H)  # 4x1x2
        corners.append(wc)
        polys.append(wc.reshape(-1,2))       # 4x2 for drawing/debug

    allc = np.vstack(corners)                # (4N,1,2)
    x_min, y_min = np.floor(allc.min(axis=0).ravel()).astype(int)
    x_max, y_max = np.ceil (allc.max(axis=0).ravel()).astype(int)

    # padding
    x_min -= PADDING
    y_min -= PADDING
    x_max += PADDING
    y_max += PADDING

    T = np.array([[1,0,-x_min],[0,1,-y_min],[0,0,1]], dtype=np.float64)
    size = (int(x_max - x_min), int(y_max - y_min))
    # --- clamp canvas to MAX_CANVAS ---
    if "MAX_CANVAS" in globals() and MAX_CANVAS is not None:
        max_dim = max(size)
        if max_dim > MAX_CANVAS:
            s = MAX_CANVAS / float(max_dim)
            S = np.array([[s,0,0],[0,s,0],[0,0,1]], dtype=np.float64)
            T = S @ T
            size = (int(size[0]*s), int(size[1]*s))
    return T, size, polys  # return polys for debug drawing


In [19]:
def _robust_mean(img, mask):
    # img: HxWx3 float32, mask: HxWx1 {0,1}
    eps = 1e-6
    m = mask.squeeze().astype(bool)
    if not np.any(m):
        return np.array([1.0, 1.0, 1.0], dtype=np.float32)
    # robust mean via median of valid pixels per channel
    vals = img[m]
    med = np.median(vals, axis=0).astype(np.float32)
    med[med < eps] = eps
    return med

def multiband_blend_warped(images, H_to_anchor, canvas_T, canvas_size, debug_show=False):
    """
    Exposure-aware streaming blender (memory-safe).
    - Per-image gain compensation on the overlap area
    - Distance-normalized feather to avoid halos
    - Same signature as your previous function
    """
    if "DEBUG" in globals() and DEBUG:
        print("[info] exposure-aware blending")

    Hs = [canvas_T @ H for H in H_to_anchor]

    Hc, Wc = canvas_size[1], canvas_size[0]
    canvas = np.zeros((Hc, Wc, 3), np.float32)
    weight = np.zeros((Hc, Wc, 1), np.float32)

    for i, (img, Hc_) in enumerate(zip(images, Hs)):
        warped = cv.warpPerspective(img, Hc_, canvas_size).astype(np.float32)
        mask = (warped.sum(axis=2, keepdims=True) > 0).astype(np.float32)

        # Overlap with existing canvas?
        overlap_mask = ((weight > 0) & (mask > 0)).astype(np.float32)

        # ---- Exposure/gain compensation (per-channel) ----
        # Compare median colors in overlap; if no overlap yet, skip.
        if np.any(overlap_mask):
            # Compute robust “means”
            cur_med  = _robust_mean(warped, overlap_mask)
            canv_med = _robust_mean(canvas / np.maximum(weight, 1e-6), overlap_mask)

            gain = (canv_med / np.maximum(cur_med, 1e-6)).astype(np.float32)
            # Clamp gains to avoid extremes
            gain = np.clip(gain, 0.8, 1.25)

            # Apply gain per channel
            warped *= gain[None, None, :]
        # else: first image into canvas → no compensation

        # ---- Distance-to-edge feather, normalized against current canvas ----
        # Distance of the *current* mask to its boundary
        dt_new = cv.distanceTransform((mask.squeeze()>0).astype(np.uint8), cv.DIST_L2, 3).astype(np.float32)
        if dt_new.max() > 0:
            dt_new /= dt_new.max()
        w_new = (0.1 + 0.9*dt_new)[..., None] * mask  # keep a floor to avoid zero-division

        # If there is already weight at those pixels, normalize so the sum stays smooth
        w_sum = w_new + weight
        # Where there is overlap, make the share proportional to each side
        w_new = np.where(w_sum > 1e-6, (w_new / w_sum), w_new)

        # Accumulate
        canvas = canvas * (1.0 - w_new) + warped * w_new
        weight = np.maximum(weight, w_new)  # track coverage/soft max (prevents runaway weight)

        if debug_show and ('DEBUG' in globals() and DEBUG):
            print(f"[blend] {i+1}/{len(images)}")

        # Free per-iter buffers
        del warped, mask, overlap_mask, dt_new, w_new, w_sum
        gc.collect()

    out = np.clip(canvas, 0, 255).astype(np.uint8)
    if debug_show and ('DEBUG' in globals() and DEBUG):
        show_img(out, "Exposure-aware mosaic")
    return out


In [20]:
import gc
MIN_RUN_LEN = 9  # require more than 8 images in a row to mosaic

def stitch_folder_split(folder_glob, out_prefix="mosaic"):
    files = sorted(glob.glob(folder_glob))
    if not files:
        raise SystemExit("No images found")

    imgs = [imread_color(f) for f in files]
    if "DEBUG" in globals() and DEBUG:
        print(f"[info] loaded {len(imgs)} images")

    # Break into subgroups whenever matches fail
    subgroups = []
    current = [imgs[0]]

    for i in range(1, len(imgs)):
        H, _, _, _ = detect_and_match(imgs[i], imgs[i-1], i, i-1)
        if H is None:
            print(f"[split] No match between {i-1} and {i} → splitting mosaic here")
            subgroups.append(current)
            current = [imgs[i]]
        else:
            current.append(imgs[i])
    subgroups.append(current)

    print(f"[info] formed {len(subgroups)} mosaics")

    # Stitch each subgroup
    subgroups = [g for g in subgroups if len(g) >= MIN_RUN_LEN]
    if 'DEBUG' in globals() and DEBUG:
        print(f"[info] kept {len(subgroups)} subgroups with >= {MIN_RUN_LEN} images")
        # Free original list to save memory
    del imgs
    gc.collect()
    for gi, group in enumerate(subgroups):
        if len(group) < 2:
            print(f"[skip] group {gi} has only 1 image")
            continue

        if "DEBUG" in globals() and DEBUG:
            print(f"[info] stitching group {gi} with {len(group)} images")
        H_to_anchor, _ = accumulate_homographies(group)
        T, canvas_size = compute_canvas_bounds(group, H_to_anchor)
        if "DEBUG" in globals() and DEBUG:
            print(f"[info] group {gi} canvas size: {canvas_size}")

        mosaic = multiband_blend_warped(group, H_to_anchor, T, canvas_size)
        out_path = f"{out_prefix}_{gi}.png"
        cv.imwrite(out_path, mosaic)
        print(f"[done] wrote {out_path}")


In [22]:
import gc
import glob
import cv2 as cv
import numpy as np

MIN_RUN_LEN = 4

files = sorted(glob.glob("Killaloe/*.jpg"))
if not files:
    raise SystemExit("No images found")

imgs = [imread_color(f) for f in files]
if "DEBUG" in globals() and DEBUG:
    print(f"[info] loaded {len(imgs)} images")

# --- split into subgroups whenever matches fail ---
subgroups = []
current = [imgs[0]]
for i in range(1, len(imgs)):
    H, _, _, _ = detect_and_match(imgs[i], imgs[i-1], i, i-1)
    if H is None:
        if "DEBUG" in globals() and DEBUG:
            print(f"[split] No match between {i-1} and {i} → new subgroup")
        subgroups.append(current)
        current = [imgs[i]]
    else:
        current.append(imgs[i])
subgroups.append(current)

print(f"[info] formed {len(subgroups)} subgroups")

# --- stitch each subgroup separately ---
subgroups = [g for g in subgroups if len(g) >= MIN_RUN_LEN]
if "DEBUG" in globals() and DEBUG:
    print(f"[info] kept {len(subgroups)} subgroups with >= {MIN_RUN_LEN} images")

# Release memory
imgs = None; gc.collect()

for gi, group in enumerate(subgroups):
    if len(group) < 2:
        print(f"[skip] subgroup {gi} has only 1 image")
        continue

    print(f"[info] stitching subgroup {gi} with {len(group)} images")

    H_to_anchor, _ = accumulate_homographies(group)
    T, canvas_size, polys = compute_canvas_bounds(group, H_to_anchor)
    if "DEBUG" in globals() and DEBUG:
        print(f"[info] subgroup {gi} canvas size: {canvas_size}")

    mosaic = multiband_blend_warped(group, H_to_anchor, T, canvas_size, debug_show=False)

    out_path = f"mosaic_{gi}.png"
    cv.imwrite(out_path, mosaic)
    print(f"[done] wrote {out_path}")


[info] formed 113 subgroups
[info] stitching subgroup 0 with 8 images


C:\Users\Luke Griffin\AppData\Local\Temp\ipykernel_50848\3039165138.py:61: RuntimeWarning: invalid value encountered in divide
  w_new = np.where(w_sum > 1e-6, (w_new / w_sum), w_new)


[done] wrote mosaic_0.png
[info] stitching subgroup 1 with 5 images
[done] wrote mosaic_1.png
[info] stitching subgroup 2 with 17 images
[done] wrote mosaic_2.png
[info] stitching subgroup 3 with 24 images
[done] wrote mosaic_3.png
[info] stitching subgroup 4 with 7 images
[done] wrote mosaic_4.png
[info] stitching subgroup 5 with 11 images
[done] wrote mosaic_5.png
[info] stitching subgroup 6 with 5 images
[done] wrote mosaic_6.png
[info] stitching subgroup 7 with 7 images
[done] wrote mosaic_7.png
[info] stitching subgroup 8 with 6 images
[done] wrote mosaic_8.png
[info] stitching subgroup 9 with 13 images
[done] wrote mosaic_9.png
